In [152]:
import pandas as pd
from bs4 import BeautifulSoup
import string
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from sklearn import preprocessing
import collections
import tqdm
import numpy as np
import os
import re
import itertools
from nltk import ngrams

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jramos/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def remove_html(text):
    soup = BeautifulSoup(text, 'lxml')
    html_free = soup.text
    return html_free

In [4]:
def buildTransitionTable(row, graph, reverse_dict):
    phrase = row['VLyric']
    #context = row['type']

    for j in range(len(phrase)):
        # Valida se a palavra atual tem vizinho
        if j+1 >= len(phrase):
            pass
        else:
            
            # Se tem vizinho, incrementa na interseção
            graph[reverse_dict[phrase[j]]][reverse_dict[phrase[j+1]]] += 1
            graph[reverse_dict[phrase[j+1]]][reverse_dict[phrase[j]]] += 1

In [5]:
def buildEmissionTable(row, emissionTable, reverseDictGenres, reverseDictWords):
    phrase = row['VLyric']
    genre = row['Genre']

    for j in range(len(phrase)):
        emission_table[reverseDictGenres[genre]][reverseDictWords[phrase[j]]] += 1

In [192]:
def remove_punctuation(text):
    clean_txt = "".join([c for c in text if c not in string.punctuation])
    return clean_txt

In [308]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [195]:
artists = pd.read_csv('artists-data.csv')
lyrics = pd.read_csv('lyrics-data.csv')

In [196]:
artists_filtered = artists[['Genre']].join(artists[['Link']].drop_duplicates())
artists_filtered = artists_filtered.loc[~artists_filtered['Link'].isnull()]

In [197]:
en_lyrics = lyrics.loc[lyrics['Idiom'] == 'ENGLISH']
en_lyrics = en_lyrics.join(artists_filtered.set_index('Link'), on='ALink')
en_lyrics = en_lyrics.loc[en_lyrics['Genre'].isin(['Rock', 'Pop', 'Hip Hop'])]
del lyrics

In [198]:
en_lyrics = pd.concat([en_lyrics[:200],en_lyrics[80000:80200],en_lyrics[-200:]])
en_lyrics.shape

(600, 6)

In [199]:
en_lyrics['Genre'].unique()

array(['Rock', 'Pop', 'Hip Hop'], dtype=object)

In [200]:
en_lyrics['Lyric'] = en_lyrics['Lyric'].str.lower()
en_lyrics = en_lyrics[['Lyric','Genre']]

In [201]:
en_lyrics['Lyric'] = en_lyrics['Lyric'].apply(lambda  text: remove_html(text))
en_lyrics['Lyric'] = en_lyrics['Lyric'].apply(lambda  text: remove_punctuation(text))

In [202]:
en_lyrics['VLyric'] = en_lyrics['Lyric'].apply(lambda  text: text.split())

In [15]:
# Transforma as frases por linha em palavras por linha
word_type = pd.DataFrame(en_lyrics['VLyric'].apply(pd.Series,1).stack())
word_type = word_type.droplevel(1)
word_type = word_type.merge(en_lyrics['Genre'].to_frame(),  left_index=True, right_index=True)

In [16]:
word_type.rename(columns={0:"words"}, inplace=True)

In [17]:
unique_words = word_type['words'].unique()
reverse_dict_words = {unique_words[i]:i for i in range(len(unique_words))}

In [18]:
transition_table = np.zeros((len(unique_words), len(unique_words)), dtype=int)
en_lyrics.apply(lambda row: buildTransitionTable(row, transition_table, reverse_dict_words), axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
189254    None
189255    None
189256    None
189257    None
189258    None
Length: 600, dtype: object

In [19]:
transition_table_norm = preprocessing.normalize(transition_table, 'l1')

In [20]:
unique_genres = en_lyrics['Genre'].unique()

In [21]:
unique_genres

array(['Rock', 'Pop', 'Hip Hop'], dtype=object)

In [22]:
reverse_dict_genre = {unique_genres[i]: i for i in range(len(unique_genres))}

In [23]:
reverse_dict_genre

{'Rock': 0, 'Pop': 1, 'Hip Hop': 2}

In [27]:
emission_table = np.zeros((len(unique_genres), len(unique_words)), dtype=int)
en_lyrics.apply(lambda row: buildEmissionTable(row, emission_table, reverse_dict_genre, reverse_dict_words), axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
189254    None
189255    None
189256    None
189257    None
189258    None
Length: 600, dtype: object

In [28]:
emission_table_norm = preprocessing.normalize(emission_table, 'l1')

In [98]:
def generateText(currentWord,
                genre,
                transitionTable, 
                emissionTable, 
                reverseDictWords, 
                reverseDictGenre):

    import numpy as np
    import math

    count = 0
    generatedText = []
    while count < 20:
        if currentWord in reverseDictWords.keys():
            currWordIdx = reverseDictWords[currentWord]
            maxProb = -math.inf
            #maxProbIdx = -1
            finalNextWord = ''
            for nextWord in reverseDictWords.keys():
                nextWordIdx = reverseDictWords[nextWord]
                transition_prob = transitionTable[currWordIdx][nextWordIdx]
                emission_prob = emissionTable[reverseDictGenre[genre]][nextWordIdx]
                final_prob = 0
                if transition_prob > 0 and emission_prob > 0:
                    final_prob = np.log(transition_prob) + np.log(emission_prob)
                    
                    if final_prob > maxProb:
                        maxProb = final_prob
                        finalNextWord = nextWord
                        
        generatedText.append(finalNextWord)       
        currentWord = finalNextWord             
        count += 1

    return generatedText


In [122]:
generateText('take', 'Rock', transition_table_norm, emission_table_norm, reverse_dict_words, reverse_dict_genre)

['the',
 'in',
 'the',
 'in',
 'the',
 'in',
 'the',
 'in',
 'the',
 'in',
 'the',
 'in',
 'the',
 'in',
 'the',
 'in',
 'the',
 'in',
 'the',
 'in']

In [113]:
reverse_dict_words['and']

64

In [119]:
transition_table_norm[64][0]

0.06053189753630282

# Otimização do Modelo

Para melhorar a precisão do modelo, aumentei o grau do grafo para que o mesmo encontre conexões diferentes entre as palavras (que são os nós do grafo).

Para aumentar o grau do grafo devemos seguir as seguintes etapas:

1) Salvar o grafo inicial

2) Multiplicar o grafo por ele mesmo

3) Somar o resultado da multiplicação com o grafo inicial

4) Normalizar o grafo final

In [51]:
transition_table_norm_rw = transition_table_norm.copy()

In [53]:
new_trans_table = np.matmul(transition_table_norm_rw, transition_table_norm_rw)

In [54]:
final_trasition_table = new_trans_table + transition_table_norm_rw
final_trasition_table = preprocessing.normalize(final_trasition_table, 'l1')

In [55]:
final_trasition_table

array([[8.40640717e-02, 4.41082406e-03, 5.59071556e-03, ...,
        2.74496180e-06, 1.83544091e-06, 3.45455554e-06],
       [1.09044274e-01, 2.18770513e-02, 7.02432351e-03, ...,
        3.65826801e-07, 0.00000000e+00, 1.24980628e-06],
       [9.76913308e-02, 4.96489427e-03, 2.03594535e-02, ...,
        1.76694168e-06, 1.42930550e-05, 0.00000000e+00],
       ...,
       [1.51329744e-02, 8.15793767e-05, 5.57470099e-04, ...,
        9.14687883e-05, 0.00000000e+00, 0.00000000e+00],
       [1.01187857e-02, 0.00000000e+00, 4.50945886e-03, ...,
        0.00000000e+00, 1.70234349e-03, 0.00000000e+00],
       [1.90449647e-02, 2.78706800e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 7.38897099e-04]])

In [58]:
unique_words

array(['i', 'could', 'feel', ..., 'mom', 'spicy', 'toilet'], dtype=object)

In [92]:
final_trasition_table[4][4]

0.10914353133622012

In [125]:
generateText('toilet', 'Rock', final_trasition_table, emission_table_norm, reverse_dict_words, reverse_dict_genre)

['that',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i']

## N-Grams

In [223]:
def makeGramCount(text, n, gram_dict):
    """ 
        This function return a dictionary with unique grams and the count of each
    """
    n_limit = n - 1
    loop_limit = len(text) - n_limit
    for i in range(loop_limit):
        current_seq = tuple(text[i:n+i])
        if current_seq in gram_dict.keys():
            gram_dict[current_seq] += 1
        else:
            gram_dict[current_seq] = 1


In [299]:
def generateNgramText(text, gram_arr, reverseDictWords, reverseDictGenre, genre, emissionTable):

    import numpy as np
    import math


    n = len(gram_arr)
    count = 0
    generatedText = []
    while count < 20:
        maxProb = -math.inf
        finalNextWord = ''
        for nextWord in reverseDictWords.keys():
            try:
                count_3 = gram_arr[2][(text[0], text[1], nextWord)]
                count_2 = gram_arr[1][(text[0], text[1])]
                count_1 = gram_arr[0][(text[0],)]
                emissionProb = emissionTable[reverseDictGenre[genre]][reverseDictWords[nextWord]]
            except KeyError:
                continue

            final_prob = np.log(count_3/count_2) + np.log(count_2/count_1) + np.log(count_1/total) + np.log(emissionProb)
            
            if final_prob > maxProb:
                maxProb = final_prob
                finalNextWord = nextWord
    
        generatedText.append(finalNextWord)  
        text[0] = text[1]
        text[1] = finalNextWord     
        currentWord = finalNextWord             
        count += 1

    return generatedText


In [294]:
unique_words[unique_words == '']

array([], dtype=object)

In [ ]:
total = 0
for w in uniGram.keys():
    total = total + uniGram[w]

In [224]:
threeGram = dict()
biGram = dict()
uniGram = dict()
en_lyrics['VLyric'].apply(lambda row: makeGramCount(row, 3, threeGram))
en_lyrics['VLyric'].apply(lambda row: makeGramCount(row, 2, biGram))
en_lyrics['VLyric'].apply(lambda row: makeGramCount(row, 1, uniGram))

0         None
1         None
2         None
3         None
4         None
          ... 
189254    None
189255    None
189256    None
189257    None
189258    None
Name: VLyric, Length: 600, dtype: object

In [318]:
generateNgramText(['being', 'taunted'], 
                  [uniGram, biGram, threeGram], 
                  reverse_dict_words, 
                  reverse_dict_genre, 
                  'Rock', 
                  emission_table_norm)

<ipython-input-299-7df786765836>:22: RuntimeWarning: divide by zero encountered in log
  final_prob = np.log(count_3/count_2) + np.log(count_2/count_1) + np.log(count_1/total) + np.log(emissionProb)


['something',
 'is',
 'out',
 'of',
 'my',
 'back',
 'you',
 'a',
 'love',
 'i',
 'never',
 'know',
 'what',
 'you',
 'see',
 'the',
 'the',
 'go',
 'and',
 'i']